In [1]:
%run common.ipynb

In [2]:
from pyspark.sql import SparkSession
import random

# Initialize Spark Session with MySQL JDBC Driver
spark = SparkSession.builder.appName("RetailCustomer360") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33") \
    .getOrCreate()


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glue_user/.ivy2/cache
The jars for the packages stored in: /home/glue_user/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d6a79007-d978-48df-8f06-2286d4379f3a;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.33 in central
	found com.mysql#mysql-connector-j;8.0.33 in central
	found com.google.protobuf#protobuf-java;3.21.9 in central
:: resolution report :: resolve 1081ms :: artifacts dl 28ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.21.9 from central in [default]
	com.mysql#mysql-connector-j;8.0.33 from central in [default]
	mysql#mysql-connector-java;8.0.33 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-----------------------------------------------------------

In [3]:
# Read MySQL table into PySpark DataFrame
customer_df = spark.read.jdbc(url=USER_MYSQL_URL, table="Customers", properties=MYSQL_PROPERTIES)
order_df = spark.read.jdbc(url=ORDER_MYSQL_URL, table="Orders", properties=MYSQL_PROPERTIES)
products_df = spark.read.jdbc(url=PRODUCT_MYSQL_URL, table="Products", properties=MYSQL_PROPERTIES)
# Register as Temp Tables
customer_df.createOrReplaceTempView("customers")
order_df.createOrReplaceTempView("orders")
products_df.createOrReplaceTempView("products")

In [8]:
customer_df.show()

+-----------+-----------+---------+--------------------+--------------------+--------------------+-------------------+-------------+--------+----------------+-------------------+
|customer_id| first_name|last_name|               email|               phone|             address|               city|        state|zip_code|         country|         created_at|
+-----------+-----------+---------+--------------------+--------------------+--------------------+-------------------+-------------+--------+----------------+-------------------+
|          1|    Michael|  Rodgers|katiecasey@exampl...|+1-664-301-3316x9...|USS Dudley\nFPO A...|        North Traci|        Idaho|   38917|     Saint Lucia|2025-03-15 07:34:22|
|          2|      Paula|    Berry|carolyn15@example...|  971-396-8026x98919|PSC 9301, Box 951...|       Hornechester|     Michigan|   14169|          Turkey|2025-03-15 07:34:32|
|          3|     Angela| Lawrence|  kchung@example.org|   616-578-2644x4229|884 Brandi Pike A...|       

In [5]:
spark.sql("""
        SELECT *   FROM orders o  where o.customer_id=553 """).show(100)

+--------+-----------+-------------------+---------+------------+
|order_id|customer_id|         order_date|   status|total_amount|
+--------+-----------+-------------------+---------+------------+
|     386|        553|2025-01-18 00:00:00|  Pending|      141.46|
|    1079|        553|2022-07-29 00:00:00|Delivered|      384.93|
|    2262|        553|2024-05-03 00:00:00|Cancelled|      165.35|
|    3996|        553|2021-02-05 00:00:00|  Shipped|       69.20|
|    5192|        553|2021-07-25 00:00:00|  Pending|      378.97|
|    5646|        553|2023-07-04 00:00:00|  Shipped|      276.54|
|    6977|        553|2023-07-30 00:00:00|  Pending|      351.98|
|    8405|        553|2022-04-12 00:00:00|  Pending|      466.65|
|    9451|        553|2022-01-01 00:00:00|Delivered|      446.68|
|    9688|        553|2023-04-30 00:00:00|  Shipped|      345.45|
|   10400|        553|2023-05-16 00:00:00|  Shipped|      315.09|
|   11051|        553|2020-02-23 00:00:00|Cancelled|      339.56|
|   11081|

In [9]:
spark.sql("""
        SELECT
            o.customer_id,
            SUM(o.total_amount) AS total_spent
        FROM orders o GROUP BY o.customer_id  order by total_spent desc""").show()

+-----------+-----------+
|customer_id|total_spent|
+-----------+-----------+
|        553|    6881.39|
|       6188|    6646.06|
|       6940|    6545.92|
|       5361|    6519.63|
|       7440|    6292.82|
|       8459|    6152.89|
|       1023|    6071.49|
|       6208|    6063.56|
|       4607|    6030.39|
|       7658|    5993.29|
|         70|    5957.60|
|       6305|    5912.17|
|       7153|    5897.98|
|       5231|    5853.98|
|       1821|    5799.72|
|       2667|    5797.99|
|       1524|    5790.05|
|       2242|    5766.12|
|       6724|    5764.37|
|       3469|    5670.20|
+-----------+-----------+
only showing top 20 rows



In [8]:
top_customers = spark.sql("""
        SELECT
            o.customer_id,
            SUM(o.total_amount) AS total_spent,
            COUNT(o.order_id) AS total_orders,
            MAX(o.order_date) AS last_purchase_date
        FROM orders o 
        WHERE o.order_date >= date_add(current_date(), -365)  -- Last 1 year
        GROUP BY o.customer_id order by total_spent desc  """).show(10)

+-----------+-----------+------------+-------------------+
|customer_id|total_spent|total_orders| last_purchase_date|
+-----------+-----------+------------+-------------------+
|        553|    6881.39|          20|2025-01-22 00:00:00|
|       6188|    6646.06|          20|2024-09-27 00:00:00|
|       6940|    6545.92|          20|2025-02-23 00:00:00|
|       5361|    6519.63|          20|2024-11-18 00:00:00|
|       7440|    6292.82|          21|2024-09-24 00:00:00|
|       8459|    6152.89|          22|2024-12-26 00:00:00|
|       1023|    6071.49|          20|2024-09-11 00:00:00|
|       6208|    6063.56|          20|2024-11-09 00:00:00|
|       4607|    6030.39|          19|2024-12-01 00:00:00|
|       7658|    5993.29|          20|2025-01-21 00:00:00|
+-----------+-----------+------------+-------------------+
only showing top 10 rows



In [13]:
 spark.sql("""
    WITH customer_spending AS (
        SELECT
            o.customer_id,
            SUM(o.total_amount) AS total_spent,
            COUNT(o.order_id) AS total_orders,
            MAX(o.order_date) AS last_purchase_date
        FROM orders o
        WHERE o.order_date >= date_add(current_date(), -365)   -- Last 1 year
        GROUP BY o.customer_id order by total_spent desc 
    )
        SELECT
            c.country,
            c.customer_id,
            c.first_name,
            c.email,
            cs.total_spent,
            cs.total_orders,
            cs.last_purchase_date,
            RANK() OVER ( PARTITION BY c.country ORDER BY cs.total_spent DESC) AS spending_rank
        FROM customer_spending cs
        JOIN customers c ON cs.customer_id = c.customer_id where c.country='United Kingdom'  GROUP BY c.country,c.customer_id,c.first_name,c.email,cs.total_spent,cs.last_purchase_date,cs.total_orders order by total_spent desc
""").show(100)


+--------------+-----------+----------+--------------------+-----------+------------+-------------------+-------------+
|       country|customer_id|first_name|               email|total_spent|total_orders| last_purchase_date|spending_rank|
+--------------+-----------+----------+--------------------+-----------+------------+-------------------+-------------+
|United Kingdom|         70| Stephanie|donnajones@exampl...|    1251.89|           4|2024-09-28 00:00:00|            1|
|United Kingdom|       2185|   Tiffany|johnsonjennifer@e...|    1234.75|           4|2025-02-18 00:00:00|            2|
|United Kingdom|       7597|     Tanya|  iwoods@example.org|    1221.33|           3|2024-12-18 00:00:00|            3|
|United Kingdom|       4895|  Jennifer|emilymanning@exam...|     917.19|           3|2024-12-14 00:00:00|            4|
|United Kingdom|       4264| Christian|tfleming@example.net|     843.86|           3|2024-09-21 00:00:00|            5|
|United Kingdom|        402|     Kevin|c

In [16]:

# Run Spark SQL Query
top_customers = spark.sql("""
    WITH customer_spending AS (
        SELECT
            o.customer_id,
            SUM(o.total_amount) AS total_spent,
            COUNT(o.order_id) AS total_orders,
            MAX(o.order_date) AS last_purchase_date
        FROM orders o
        WHERE o.order_date >= date_add(current_date(), -365)  -- Last 1 year
        GROUP BY o.customer_id
    ),
    customer_ranking AS (
        SELECT
            c.country,
            c.customer_id,
            c.first_name,
            c.email,
            cs.total_spent,
            cs.total_orders,
            cs.last_purchase_date,
            RANK() OVER (PARTITION BY c.country ORDER BY cs.total_spent DESC) AS spending_rank
        FROM customer_spending cs
        JOIN customers c ON cs.customer_id = c.customer_id
    )
    SELECT * FROM customer_ranking WHERE spending_rank <= 10 and country like 'United %';
""")

# Show results
top_customers.show(100)


+--------------------+-----------+----------+--------------------+-----------+------------+-------------------+-------------+
|             country|customer_id|first_name|               email|total_spent|total_orders| last_purchase_date|spending_rank|
+--------------------+-----------+----------+--------------------+-----------+------------+-------------------+-------------+
|United Arab Emirates|       5354|      Ryan|lisamorrison@exam...|    1565.06|           5|2025-02-27 00:00:00|            1|
|United Arab Emirates|       6218| Katherine|walkerrobert@exam...|    1473.88|           4|2024-10-15 00:00:00|            2|
|United Arab Emirates|       8647|  Danielle|  bsmith@example.com|    1367.50|           5|2025-02-11 00:00:00|            3|
|United Arab Emirates|        110|  Samantha|stephenobrien@exa...|    1218.35|           4|2024-11-09 00:00:00|            4|
|United Arab Emirates|       5727|     Kevin| david25@example.com|    1153.08|           4|2025-01-25 00:00:00|       

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window


# Filter last 1 year of data
one_year_ago = F.date_add(F.current_date(), -365)
filtered_orders = order_df.filter(F.col("order_date") >= one_year_ago)

# Aggregate customer spending
customer_spending = (
    filtered_orders.groupBy("customer_id")
    .agg(
        F.sum("total_amount").alias("total_spent"),
        F.count("order_id").alias("total_orders"),
        F.max("order_date").alias("last_purchase_date")
    )
)

# Join with customers table
customer_data = customer_spending.join(customer_df, "customer_id")

# Define window specification for ranking
window_spec = Window.partitionBy("country").orderBy(F.desc("total_spent"))

# Add ranking column
customer_ranking = customer_data.withColumn("spending_rank", F.rank().over(window_spec))

# Filter top 100 customers
top_customers = customer_ranking.filter( (F.col("spending_rank") <= 10) &  (F.col("country").like("United %")))

# Show results
top_customers.select("country","customer_id","first_name","email","total_spent","total_orders","spending_rank").show(20)

In [17]:
# Define AWS Glue database and table names
glue_database = "customer_analytics"
glue_table = "top_100_customers"

# Define S3 output path
s3_output_path = "s3://feb2025-training-bucket/analytics/top_100_customers/"

# Create the AWS Glue Catalog table using the DataFrame
top_customers.write \
    .format("parquet") \
    .mode("overwrite") \
    .option("path", s3_output_path) \
    .saveAsTable(f"{glue_database}.{glue_table}")

print(f"Aggregated sales data written to S3: {s3_output_path}")
print(f"Glue table '{glue_database}.{glue_table}' created successfully.")


25/03/13 04:38:47 INFO HiveConf: Found configuration file file:/home/glue_user/spark/conf/hive-site.xml
25/03/13 04:38:48 WARN InstanceMetadataServiceResourceFetcher: Fail to retrieve token 
com.amazonaws.SdkClientException: Failed to connect to service endpoint: 
	at com.amazonaws.internal.EC2ResourceFetcher.doReadResource(EC2ResourceFetcher.java:100)
	at com.amazonaws.internal.InstanceMetadataServiceResourceFetcher.getToken(InstanceMetadataServiceResourceFetcher.java:91)
	at com.amazonaws.internal.InstanceMetadataServiceResourceFetcher.readResource(InstanceMetadataServiceResourceFetcher.java:69)
	at com.amazonaws.internal.EC2ResourceFetcher.readResource(EC2ResourceFetcher.java:66)
	at com.amazonaws.util.EC2MetadataUtils.getItems(EC2MetadataUtils.java:407)
	at com.amazonaws.util.EC2MetadataUtils.getData(EC2MetadataUtils.java:376)
	at com.amazonaws.util.EC2MetadataUtils.getData(EC2MetadataUtils.java:372)
	at com.amazonaws.util.EC2MetadataUtils.getEC2InstanceRegion(EC2MetadataUtils.java

Aggregated sales data written to S3: s3://feb2025-training-bucket/analytics/top_100_customers/
Glue table 'customer_analytics.top_100_customers' created successfully.
